In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server
import bnpm

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
from tqdm import tqdm
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [8]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username: rh183
Password: ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
# ## initialize ssh_compute
# ssh_c = server.ssh_interface(
#     nbytes_toReceive=20000,
#     recv_timeout=1,
#     verbose=True,
# )
# ssh_c.o2_connect(
#     hostname=remote_host_compute,
#     username=username,
#     password=server.pw_decode(pw),
#     key_filename=path_sshKey,
#     look_for_keys=False,
#     passcode_method=1,
#     verbose=0,
#     skip_passcode=False,    
# )

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
# send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

In [12]:
cage = 'cage_0403'
mouse = 'mouse_0403R'

# Download stat and ops files

In [87]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/{mouse}/',
    search_pattern_re='stat.*npy',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231205/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231029/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231127/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231024/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231021/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231025/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231106/scanimage_data/exp/jobNum_0/suite2p/plane0/stat.npy
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0916N/20231

In [88]:
paths_found = [p for p in paths_found if 'exp' in p]

In [89]:
dates_toUse = natsort.natsorted([Path(p).parts[-7] for p in paths_found])
dates_toUse

['20231021',
 '20231022',
 '20231023',
 '20231024',
 '20231025',
 '20231026',
 '20231027',
 '20231028',
 '20231029',
 '20231030',
 '20231031',
 '20231101',
 '20231102',
 '20231103',
 '20231104',
 '20231105',
 '20231106',
 '20231127',
 '20231128',
 '20231129',
 '20231130',
 '20231201',
 '20231202',
 '20231203',
 '20231204',
 '20231205',
 '20231207',
 '20231208',
 '20231209',
 '20231210']

In [84]:
[sftp.get(str(Path(p).parent / 'stat.npy'), str(Path(f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/stat_and_ops') / Path(p).parts[-7] / 'stat.npy')) for p in paths_found[:]];

100%|█████████████████████████████████████████████████████████████████████▉| 16.5M/16.5M [00:00<00:00, 43.4Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 18.1M/18.1M [00:00<00:00, 51.5Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 16.4M/16.4M [00:00<00:00, 49.5Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 18.4M/18.4M [00:00<00:00, 52.1Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 18.5M/18.5M [00:00<00:00, 48.9Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 18.2M/18.2M [00:00<00:00, 32.7Mb/s]


In [90]:
[sftp.get(str(Path(p).parent / 'ops.npy'), str(Path(f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/stat_and_ops') / Path(p).parts[-7] / 'ops.npy')) for p in paths_found[:]];

100%|███████████████████████████████████████████████████████████████████████▉| 232M/232M [00:03<00:00, 59.7Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 232M/232M [00:03<00:00, 63.6Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 231M/231M [00:03<00:00, 63.3Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 230M/230M [00:03<00:00, 61.9Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 229M/229M [00:03<00:00, 61.9Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 230M/230M [00:04<00:00, 54.2Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 230M/230M [00:03<00:00, 63.2Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 232M/232M [00:03<00:00, 62.7Mb/s]
100%|███████████████████████████████████████████████████████████████████████▉| 230M/230M [00:03<

# Download logger files

In [14]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path=f'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/{cage}/{mouse}/',
    search_pattern_re='logger\.mat',
    max_depth=7,
    find_folders=False,
    find_files=True,
    verbose=True,
))

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230708/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230709/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230702/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230703/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230707/analysis_data/logger.mat
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230706/analysis_data/logger.mat


In [15]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230702', '20230703', '20230706', '20230707', '20230708', '20230709']

In [16]:
[sftp.get(str(Path(p).parent / 'logger.mat'), str(Path(f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/loggers') / date / 'logger.mat')) for date, p in zip(dates_toUse, paths_found)];

100%|█████████████████████████████████████████████████████████████████████▉| 4.69M/4.69M [00:00<00:00, 17.3Mb/s]
100%|█████████████████████████████████████████████████████████████████████▊| 4.65M/4.67M [00:00<00:00, 19.9Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 9.54M/9.55M [00:00<00:00, 26.0Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 7.50M/7.51M [00:00<00:00, 24.9Mb/s]
100%|█████████████████████████████████████████████████████████████████████▊| 10.6M/10.6M [00:00<00:00, 28.6Mb/s]
100%|█████████████████████████████████████████████████████████████████████▊| 10.6M/10.6M [00:00<00:00, 30.3Mb/s]


In [17]:
[sftp.get(str(Path(p).parent / 'expParams.mat'), str(Path(f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/loggers') / date / 'expParams.mat')) for date, p in zip(dates_toUse, paths_found)];

 95%|██████████████████████████████████████████████████████████████████▊   | 65.5k/68.6k [00:00<00:00, 1.15Mb/s]
 95%|██████████████████████████████████████████████████████████████████▊   | 65.5k/68.6k [00:00<00:00, 1.07Mb/s]
 99%|█████████████████████████████████████████████████████████████████████▌| 5.24M/5.27M [00:00<00:00, 25.0Mb/s]
100%|█████████████████████████████████████████████████████████████████████▊| 5.37M/5.39M [00:00<00:00, 24.9Mb/s]
 99%|█████████████████████████████████████████████████████████████████████▌| 5.34M/5.37M [00:00<00:00, 25.0Mb/s]
100%|█████████████████████████████████████████████████████████████████████▊| 5.21M/5.22M [00:00<00:00, 25.7Mb/s]


# Download wavesurfer files

In [64]:
paths_files_remote = sftp.search_recursive(
    path=f'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/{cage}/{mouse}/',
    search_pattern_re=r'.*exp.*h5',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_files_remote = natsort.natsorted(paths_files_remote)

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230708/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230709/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230702/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230703/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230707/wavesurfer_data/exp_0001.h5
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230706/wavesurfer_data/exp_0001.h5


In [65]:
dates_files_remote = [Path(p).parent.parent.name for p in paths_files_remote]

In [66]:
dates_files_remote

['20230702', '20230703', '20230706', '20230707', '20230708', '20230709']

In [67]:
directory_local_save = f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/wavesurfer_files/'

In [68]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_files_remote, dates_files_remote)]

100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 50.9Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 56.4Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 58.8Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 60.9Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 53.2Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 50.7M/50.7M [00:00<00:00, 56.5Mb/s]


[None, None, None, None, None, None]

# Download camera CSV files

In [44]:
paths_csvs_remote = sftp.search_recursive(
    path=f'/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/{cage}/{mouse}/',
    search_pattern_re=r'.*cam4.*csv',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
)

paths_csvs_remote = natsort.natsorted(paths_csvs_remote)

/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230708/camera_data/exp/cam4/times_cam42023-07-08T12_17_59.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230709/camera_data/exp/cam4/times_cam42023-07-09T12_19_41.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230702/camera_data/exp/cam4/times_cam42023-07-02T14_46_44.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230703/camera_data/exp/cam4/times_cam42023-07-03T11_31_50.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230707/camera_data/exp/cam4/times_cam42023-07-07T12_09_12.csv
/n/files/Neurobio/MICROSCOPE/Gyu/BMI_data/cage_0403/mouse_0403R/20230706/camera_data/exp/cam4/times_cam42023-07-06T13_45_36.csv


In [45]:
dates_csvs_remote = [Path(p).parent.parent.parent.parent.name for p in paths_csvs_remote]

In [46]:
directory_local_save = f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/cam4_CSVs/'

In [47]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_csvs_remote)];

100%|█████████████████████████████████████████████████████████████████████▉| 31.1M/31.1M [00:01<00:00, 31.0Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 34.8M/34.8M [00:00<00:00, 35.1Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 31.2M/31.2M [00:00<00:00, 39.4Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 42.7Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 31.1M/31.1M [00:00<00:00, 31.3Mb/s]
100%|█████████████████████████████████████████████████████████████████████▉| 31.2M/31.3M [00:01<00:00, 29.5Mb/s]


# Download face-rhythm files

In [13]:
paths_fr_remote = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/{mouse}/',
    search_pattern_re=r'VQT_Analyzer.h5',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230702/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230707/jobNum_0/analysis_files/VQT_Analyzer.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706/jobNum_0/analysis_files/VQT_Analyzer.h5


In [14]:
dates_fr_remote = [Path(p).parts[-4] for p in paths_fr_remote]
dates_fr_remote

['20230702', '20230706', '20230707', '20230708', '20230709']

In [15]:
dirsToDownload_fr_remote = [str(Path(p).parent.parent.parent) for p in paths_fr_remote]
dirsToDownload_fr_remote

['/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230702',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230707',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708',
 '/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230709']

In [16]:
directory_local_save = f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/fr_run/'

In [17]:
directory_local_save

'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/'

In [18]:
sftp.get_dir(source=f'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/{mouse}/', target=directory_local_save)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/run_20240131/20230708/parameters_batch.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/run_20240131/20230708/jobNum_0/params.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230708/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/run_20240131/20230708/jobNum_0/config.yaml


...

0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/run_20240131/20230706/notes.txt


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0403R/run_20240131/20230706/run_pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/fr_run/run_20240131/20230706/run_pipeline_basic.py


0.00b [00:00, ?b/s]


##### 

# Upload eye laser masks

In [19]:
sftp.put_dir(
    source=f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/eye_laser_trace_extraction/',
    target=f'/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/{mouse}/masks',
)

uploading /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/20230702/mask_cam42023-07-02T14_46_45.npy   to   /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230702/mask_cam42023-07-02T14_46_45.npy
uploading /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/20230703/mask_cam42023-07-03T11_31_50.npy   to   /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230703/mask_cam42023-07-03T11_31_50.npy
uploading /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/20230706/mask_cam42023-07-06T13_45_37.npy   to   /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230706/mask_cam42023-07-06T13_45_37.npy
uploading /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/20230707/mask_cam42023-07-07T12_09_13.npy   to   /n/data1/hms/neurobio/sabatini/r

# Download eye laser traces

In [31]:
sftp.get_dir(
    source=f'/n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/{mouse}',
    target=f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/eye_laser_trace_extraction/traces',
)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/traces/20230708/parameters_batch.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/extract_eye_brightness_trace.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/traces/20230708/extract_eye_brightness_trace.py


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/20230708/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/traces/20230708/jobNum_0/params.json


...

0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230707/mask_cam42023-07-07T12_09_13.npy   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/traces/masks/20230707/mask_cam42023-07-07T12_09_13.npy


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/eye_laser_trace_extraction/mouse_0403R/masks/20230706/mask_cam42023-07-06T13_45_37.npy   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/eye_laser_trace_extraction/traces/masks/20230706/mask_cam42023-07-06T13_45_37.npy


0.00b [00:00, ?b/s]


# Download dFoF and tqm files

In [29]:
dir_dFoF_and_tqm_remote = f'/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/{mouse}'
directory_local_save = f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/dFoF_and_tqm/'

In [30]:
sftp.get_dir(source=dir_dFoF_and_tqm_remote, target=directory_local_save)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/dFoF_and_tqm/20230708/parameters_batch.json


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708/jobNum_0/tqm.pkl   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/dFoF_and_tqm/20230708/jobNum_0/tqm.pkl


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230708/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/dFoF_and_tqm/20230708/jobNum_0/params.json


...

0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230706/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/dFoF_and_tqm/20230706/dispatcher.py


0.00b [00:00, ?b/s]


downloading /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403R/20230706/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0403R/dFoF_and_tqm/20230706/notes.txt


0.00b [00:00, ?b/s]


In [ ]:
paths_dFoF_remote = natsort.natsorted(sftp.search_recursive(
    path=f'/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/{mouse}',
    search_pattern_re=r'dFoF.npy',
    max_depth=6,
    find_files=True,
    find_folders=False,
    verbose=True,
))

In [42]:
dates_dFoF_remote = [Path(p).parts[-3] for p in paths_dFoF_remote]

In [43]:
dates_dFoF_remote

['20230419',
 '20230420',
 '20230424',
 '20230425',
 '20230426',
 '20230427',
 '20230428',
 '20230429',
 '20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230519',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230530',
 '20230531']

In [44]:
directory_local_save = f'/media/rich/bigSSD/analysis_data/face_rhythm/{mouse}/dFoF_and_tqm/'

In [ ]:
[sftp.get(remotepath=p_r, localpath=str(Path(directory_local_save) / date / Path(p_r).name)) for p_r, date in zip(paths_csvs_remote, dates_dFoF_remote)];

In [ ]:
[sftp.get(remotepath=str(Path(p_r).parent / 'iscell_tqm.npy'), localpath=str(Path(directory_local_save) / date / 'iscell_tqm.npy')) for p_r, date in zip(paths_csvs_remote, dates_dFoF_remote)];

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()